# LwF

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import LwF
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 20
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/lwf.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'alpha': 0.7,
    'temperature': 20,
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loss_metrics(minibatch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(minibatch=True, experience=True, stream=True),
    gpu_usage_metrics(experience=True, stream=True, gpu_id=TORCH_DEVICE),
    disk_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True, stream=True),
    accuracy_matrix_metrics(),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(stream=True, num_classes=scenario.n_classes),
    
    loggers=[InteractiveLogger()]
)

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")
/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")


## Plugin defitinitions

In [10]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [11]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = LwF(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [12]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/20 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:08<00:00, 52.99it/s]
Epoch 0 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 120.4000
	Loss_MB/train_phase/train_stream/Task000 = 1.0117
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 8.8493
100%|██████████| 469/469 [00:09<00:00, 47.64it/s]
Epoch 1 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 125.8000
	Loss_MB/train_phase/train_stream/Task000 = 0.6802
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 9.8535
100%|██████████| 469/469 [00:11<00:00, 41.12it/s]
Epoch 2 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 102.6000
	Loss_MB/train_phase/train_stream/Task000 = 0.6425
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 11.4062
100%|██████████| 469/469 [00:11<00:00, 41.54it/s]
Epoch 3 ended.
	CPUUsage_MB/train_phase/train_str

  5%|▌         | 1/20 [01:57<37:19, 117.86s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 56.6303
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x14de1e130>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9102, 0.1142, 0.1091, 0.1089, 0.1147, 0.1039, 0.1285, 0.1279, 0.1040,
         0.1117, 0.1049, 0.1411, 0.1209, 0.1523, 0.1442, 0.0950, 0.1064, 0.1344,
         0.1141, 0.1567],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

 10%|█         | 2/20 [05:04<47:32, 158.47s/it]

...

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 48.8900
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x165200eb0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9102, 0.1142, 0.1091, 0.1089, 0.1147, 0.1039, 0.1285, 0.1279, 0.1040,
         0.1117, 0.1049, 0.1411, 0.1209, 0.1523, 0.1442, 0.0950, 0.1064, 0.1344,
         0.1141, 0.1567],
        [0.8954, 0.9269, 0.1398, 0.1032, 0.1352, 0.0856, 0.0942, 0.1000, 0.1168,
         0.1380, 0.1173, 0.1436, 0.1433, 0.1219, 0.1806, 0.0822, 0.0727, 0.1089,
         0.1028, 0.1179],
        [0.8787, 0.8992, 0.9326, 0.0928, 0.0932, 0.1075, 0.0823, 0.1079, 0.0972,
         0.1163, 0.0927, 0.1409, 0.1552, 0.1009, 0.1851, 0.0588, 0.0626, 0.1390,
         0.1252, 0.0946],
        [0.8430, 0.8746, 0.9132, 0.9338, 0.0729, 0.0743, 0.1070, 0.0911, 0.0934,
         0.1105, 0.0932, 0.1713, 0.1771, 0.1305, 0.20

 45%|████▌     | 9/20 [28:00<38:18, 208.97s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 42.9443
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x165a216a0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9102, 0.1142, 0.1091, 0.1089, 0.1147, 0.1039, 0.1285, 0.1279, 0.1040,
         0.1117, 0.1049, 0.1411, 0.1209, 0.1523, 0.1442, 0.0950, 0.1064, 0.1344,
         0.1141, 0.1567],
        [0.8954, 0.9269, 0.1398, 0.1032, 0.1352, 0.0856, 0.0942, 0.1000, 0.1168,
         0.1380, 0.1173, 0.1436, 0.1433, 0.1219, 0.1806, 0.0822, 0.0727, 0.1089,
         0.1028, 0.1179],
        [0.8787, 0.8992, 0.9326, 0.0928, 0.0932, 0.1075, 0.0823, 0.1079, 0.0972,
         0.1163, 0.0927, 0.1409, 0.1552, 0.1009, 0.1851, 0.0588, 0.0626, 0.1390,
         0.1252, 0.0946],
        [0.8430, 0.8746, 0.9132, 0.9338, 0.0729, 0.0743, 0.1070, 0.0911, 0.0934,
         0.1105, 0.0932, 0.1713, 0.1771, 0.1305, 0.20

 50%|█████     | 10/20 [30:47<32:40, 196.04s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 62.3856
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x16621a220>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9102, 0.1142, 0.1091, 0.1089, 0.1147, 0.1039, 0.1285, 0.1279, 0.1040,
         0.1117, 0.1049, 0.1411, 0.1209, 0.1523, 0.1442, 0.0950, 0.1064, 0.1344,
         0.1141, 0.1567],
        [0.8954, 0.9269, 0.1398, 0.1032, 0.1352, 0.0856, 0.0942, 0.1000, 0.1168,
         0.1380, 0.1173, 0.1436, 0.1433, 0.1219, 0.1806, 0.0822, 0.0727, 0.1089,
         0.1028, 0.1179],
        [0.8787, 0.8992, 0.9326, 0.0928, 0.0932, 0.1075, 0.0823, 0.1079, 0.0972,
         0.1163, 0.0927, 0.1409, 0.1552, 0.1009, 0.1851, 0.0588, 0.0626, 0.1390,
         0.1252, 0.0946],
        [0.8430, 0.8746, 0.9132, 0.9338, 0.0729, 0.0743, 0.1070, 0.0911, 0.0934,
         0.1105, 0.0932, 0.1713, 0.1771, 0.1305, 0.20

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    class_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/{i}"
    ][1]
    losses[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Loss_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

In [ ]:
train_cpu_usage = eval_plugin.get_all_metrics()["CPUUsage_MB/train_phase/train_stream/Task000"][1]
train_time_epoch = eval_plugin.get_all_metrics()["Time_Epoch/train_phase/train_stream/Task000"][1]
train_running_time = eval_plugin.get_all_metrics()["RunningTime_Epoch/train_phase/train_stream/Task000"][1]
train_loss_epoch = eval_plugin.get_all_metrics()["Loss_MB/train_phase/train_stream/Task000"][1]

In [ ]:
weights = model_plugins[0].weights

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
    "class_accuracies": class_accuracies,
    "losses": losses,
    "train_cpu_usage": train_cpu_usage,
    "train_time_epoch": train_time_epoch,
    "train_running_time": train_running_time,
    "train_loss_epoch": train_loss_epoch,
    "weights": weights
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))